In [ ]:
# importing the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
import sklearn.metrics as metrics
from IPython.display import display, HTML, display_html
import datetime


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Reading the Data 
data = pd.read_csv ("/content/drive/MyDrive/ads_inactive.csv",error_bad_lines=False, sep=';')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:

# look at data
display(HTML('<b>Head Data</b>'))
display(data.head())


,_id,ad_crawling_date,ad_price,ad_source,ad_title,car_energy,car_km,car_transmission,resulted_from,car_brand,...,ad_inactive,currency,cluster_key,is_clustered,cluster_kp_score,cluster_kp_scored_at,is_kp_scored,country_code,saler_type,ad_lifetime
0,60980a3f7d3e012193cdbad2,20:07.2,89900,https://www.reezocar.com,volkswagen touareg 3.0 tsi 462 ch hybride rech...,hybride,5000,automatique,scrap,volkswagen,...,True,EUR,e2444100d9111a81d91620adfbc4cde976288095,True,999,48:41.5,True,FR,private,51
1,60980a3f7d3e012193cdbad3,19:00.0,32990,https://www.reezocar.com,volkswagen touareg 2 3.0 v6 tdi 262 r-line tip...,diesel,77549,automatique,scrap,volkswagen,...,True,EUR,8730c55a177da1daa7b4733936d985ae6dd9f613,True,999,25:47.9,True,FR,private,12
2,60980a3f7d3e012193cdbad4,56:37.0,27990,https://www.reezocar.com,volkswagen touareg 3.0 v6 tdi 245ch bluemotion...,diesel,86800,automatique,scrap,volkswagen,...,True,EUR,8730c55a177da1daa7b4733936d985ae6dd9f613,True,999,25:47.9,True,FR,private,9
3,60980a3f7d3e012193cdbad5,19:00.3,29950,https://www.reezocar.com,volkswagen touareg 3.otdi 2o4cv airmatic panor...,diesel,69425,automatique,scrap,volkswagen,...,True,EUR,8730c55a177da1daa7b4733936d985ae6dd9f613,True,999,25:47.9,True,BE,private,12
4,60980a3f7d3e012193cdbad6,56:34.3,26500,https://www.reezocar.com,volkswagen touareg touareg 3.0 v6 tdi 204 tipt...,diesel,45051,manuelle,scrap,volkswagen,...,True,EUR,fa923f52149e5227d48a4758649ab1a7a93926a5,True,9,45:39.9,True,FR,private,9


In [ ]:
# look a shape of data
display(HTML('<b>Dataframe Shape</b>'))
display(data.shape)

(100000, 26)

In [ ]:
# look at data types. Ideally look at all rows. 
display(HTML('<b>Data Types</b>'))
display(data.iloc[:,:].dtypes)

_id                     object
ad_crawling_date        object
ad_price                 int64
ad_source               object
ad_title                object
car_energy              object
car_km                   int64
car_transmission        object
resulted_from           object
car_brand               object
car_color               object
car_model               object
car_real_power          object
owner_location          object
ad_creation_date        object
ad_active                 bool
ad_inactive               bool
currency                object
cluster_key             object
is_clustered              bool
cluster_kp_score         int64
cluster_kp_scored_at    object
is_kp_scored              bool
country_code            object
saler_type              object
ad_lifetime              int64
dtype: object

In [ ]:
# see if any columns have nulls. Ideally look at all rows. 
display(HTML('<b>Nulls</b>'))
display(data.iloc[:,:].isnull().any())

_id                     False
ad_crawling_date        False
ad_price                False
ad_source               False
ad_title                False
car_energy              False
car_km                  False
car_transmission        False
resulted_from           False
car_brand               False
car_color                True
car_model               False
car_real_power           True
owner_location           True
ad_creation_date        False
ad_active               False
ad_inactive             False
currency                False
cluster_key             False
is_clustered            False
cluster_kp_score        False
cluster_kp_scored_at     True
is_kp_scored            False
country_code             True
saler_type               True
ad_lifetime             False
dtype: bool

In [ ]:
# display descriptive statistics
display(HTML('<b>Describe</b>'))
display(data.describe(percentiles=[0.25,0.5,0.75,0.85,0.95,0.99]))

,ad_price,car_km,cluster_kp_score,ad_lifetime
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,33880.532420,62028.638650,756.298790,82.066040
std,46880.571012,63425.857637,425.877963,69.459266
min,137.000000,0.000000,0.000000,9.000000
25%,12900.000000,11000.000000,999.000000,28.000000
50%,21994.000000,44800.000000,999.000000,60.000000
75%,37852.500000,93000.000000,999.000000,154.000000
85%,51602.000000,125950.900000,999.000000,185.000000
95%,97000.000000,186000.000000,999.000000,204.000000
99%,228585.170000,264000.000000,999.000000,215.000000


In [ ]:
# display duplicated index. You may check other columns too
display(HTML('<b>Duplicated</b>'))
data[data.index.duplicated(keep='last')]
#df[df.duplicated('column', keep='last')]

,_id,ad_crawling_date,ad_price,ad_source,ad_title,car_energy,car_km,car_transmission,resulted_from,car_brand,...,ad_inactive,currency,cluster_key,is_clustered,cluster_kp_score,cluster_kp_scored_at,is_kp_scored,country_code,saler_type,ad_lifetime


In [ ]:
#set correct index
data.index

RangeIndex(start=0, stop=100000, step=1)

In [ ]:
data.drop(['_id','ad_crawling_date','car_brand','ad_source','ad_title','car_energy','car_transmission','resulted_from','car_color','car_model', 'owner_location','ad_creation_date','ad_active','ad_inactive','currency','cluster_key','is_clustered','cluster_kp_score','cluster_kp_scored_at','is_kp_scored','country_code','saler_type'] ,axis=1, inplace=True)

#drop_multiple_col("",data)

In [ ]:
data.drop('car_real_power',axis=1,inplace=True)

In [ ]:
#check missing data
def check_missing_data(data):
    # check for any missing data in the df (display in descending order)
    return data.isnull().sum().sort_values(ascending=False)

check_missing_data(data)

ad_price       0
car_km         0
ad_lifetime    0
dtype: int64

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   ad_price     100000 non-null  int64
 1   car_km       100000 non-null  int64
 2   ad_lifetime  100000 non-null  int64
dtypes: int64(3)
memory usage: 2.3 MB


In [ ]:
X = data.loc[:, data.columns != 'ad_lifetime']
X.head()


,ad_price,car_km
0,89900,5000
1,32990,77549
2,27990,86800
3,29950,69425
4,26500,45051


In [ ]:

y= data['ad_lifetime'].values
#print(X)
y

array([51, 12,  9, ..., 67, 35, 21])

In [ ]:
print(X.shape)
print(y.shape)

if X.shape[0] != y.shape[0]:
  print("X and y rows are mismatched, check dataset again")

(100000, 2)
(100000,)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=0) 

In [ ]:
regressor = LinearRegression()  
regressor.fit(X_train, Y_train) 

print("Training complete.")

Training complete.


In [ ]:
print(Y_test) # Testing data - In Hours
Y_pred = regressor.predict(X_test) # Predicting the ad_lifetime
print(Y_pred)

[ 35   9 184 ...   9  98 154]
[78.98943625 73.41892072 76.42378202 ... 73.71798159 82.00311595
 85.55363467]


In [ ]:
# Comparing Actual vs Predicted
data= pd.DataFrame({'Actual': Y_test, 'Predicted': Y_pred})  
data

,Actual,Predicted
0,35,78.989436
1,9,73.418921
2,184,76.423782
3,27,81.014816
4,97,74.428946
...,...,...
19995,184,76.001933
19996,80,77.387914
19997,9,73.717982
19998,98,82.003116


In [ ]:

print('Mean Absolute Error:', 
      metrics.mean_absolute_error(Y_test, Y_pred)) 

Mean Absolute Error: 57.995303080893656
